In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
import copy

In [ ]:
# R and Q are the covariance of the motion and observation models
R = 0.2
Q = 0.2
graphics_radius = 0.1

x_est = np.array([0, 1.5, 2.4, 3.4])
obs = np.array([ 2.9,  2.0, 1.0, 0.0 ])
x_true = np.array([0, 1, 2, 3])
landmark = 3
assert len(x_est) == len(obs) == len(x_true) 
N = len(x_est)

# Omega is the information matrix, which is used in negative_log_likelihood = error^T * Omega * error 
# So to minimize the max likelihood of the pose trajectory, we want to minimize negative_log_likelihood
# for likelihood, we assume the error is Gaussian: e*e/sigma^2, so to reflect that, Omega = [1/sigma^2] 
# in this case
omega = 1/Q

# counting combinations is enough for calculating the cost between all combos
node_pairs = list(itertools.combinations(range(N), 2))
for _ in range(4):
    # e_ij = z_j_hat - z_i_hat, where z_i_hat = x_i + obs_i
    H = np.zeros((N, N))
    b = np.zeros((N, 1))
    for i, j in node_pairs:
        # error is the difference between world frame observations from two nodes
        e_ij = x_est[j] + obs[j] - (x_est[i] + obs[i])
        # Jacobian of error for node i and j, with respect to all changes in pose estimates:
        # j_ij = [..., -1, ... 1, ...], 
        # b_ij = e_ij ^ T [Q] J_ij 
        J_ij = np.zeros((N,1))
        J_ij[i] = -1
        J_ij[j] = 1
        b_ij = e_ij * omega * J_ij
        b += b_ij
        
        H_ij = np.zeros((N, N))
        # so H = J^T Q J => H_ii = Q = H_jj; H_ij = -Q; H_ji = -Q
        H_ij[i, i] = H_ij[j, j] = omega
        H_ij[i, j] = H_ij[j, i] = -omega
        H += H_ij
    # Important: adding H[0,0] = 100 to assert 
    # "any delta x from node 0 to itself will be amplified large, because we are confident about node 0"
    H[0,0] += 100
    #TODO Remember to remove
    print(f'Rico: ===============')
    print(f'H : {H}')
    print(f'b: {b}')
    # Note, it's dx = H^-1 * -b
    dx = np.linalg.inv(H) @ (-b)
    x_est += dx.flatten()
    print(f'x_est: {x_est}')



In [108]:
##########################################################
# Utils
##########################################################

def normalize_to_pi(theta: float):
    return np.arctan2(np.sin(theta), np.cos(theta))


def get_A(x_odom, x_obs, i):
    x_i, y_i, theta_i = x_odom[i]
    d_i, psi_i = x_obs[i]
    #TODO Remember to remove
    print(f'Rico: d_i = {d_i}, psi_i = {psi_i}')
    A_ij = np.array([
        [-1, 0, d_i * np.sin(theta_i + psi_i)],
        [0, -1, -d_i * np.cos(theta_i + psi_i)]
    ])
    return A_ij

def get_B(x_odom, x_obs, j):
    return -get_A(x_odom=x_odom, x_obs=x_obs, i=j)

def get_J_error(x_odom, i, range, bearing):
    # This is to take the derivative of
    # z = [ x+d*cos(theta + psi), y + d*sin(theta + psi)]
    # with regards to range and bearing. 
    x_i, y_i, theta_i = x_odom[i]
    theta = theta_i + bearing
    theta = normalize_to_pi(theta)
    return np.array([
        [np.cos(theta), -range * np.sin(theta)],
        [np.sin(theta), range * np.cos(theta)],
    ])

def get_e(x_odom, x_obs, i, j):
    x_i, y_i, theta_i = x_odom[i]
    x_j, y_j, theta_j = x_odom[j]
    d_i, psi_i = x_obs[i]
    d_j, psi_j = x_obs[j]
    e_ij = np.array([
        x_j+d_j*np.cos(theta_j + psi_j) - (x_i+d_i*np.cos(theta_i + psi_i)),
        y_j + d_j*np.sin(theta_j + psi_j) - (y_i + d_i*np.sin(theta_i + psi_i)) 
    ])
    return e_ij

def get_rotational_matrix(theta):
    return np.array([
        [np.cos(theta), -np.sin(theta)],
        [np.sin(theta), np.cos(theta)]
    ])

def get_observation(landmark_x, landmark_y, x_true):
    x_ob = []
    for true_pose in x_true:
        range = np.sqrt((landmark_x - true_pose[0])**2 + (landmark_y - true_pose[1])**2)
        bearing = np.arctan2(landmark_y - true_pose[1], landmark_x - true_pose[0]) - true_pose[2]
        x_ob.append([range, bearing])
    return np.asarray(x_ob)

def get_omega(x_odom, x_obs, i, j, Q):
    x_i, y_i, theta_i = x_odom[i]
    x_j, y_j, theta_j = x_odom[j]
    d_i, psi_i = x_obs[i]
    d_j, psi_j = x_obs[j]
    RTi = get_rotational_matrix(theta_i + psi_i)
    RTj = get_rotational_matrix(theta_j + psi_j)
    return np.linalg.inv(RTi@Q@RTi.T + RTj@Q@RTj.T)
    # return np.linalg.inv(Q)
    

## Graph SLAM With 2 2D Landmark 
we have trajectory composed of three poses in (x, y, theta): (0, 0, 0), (1, 0, 0), (1,1, pi/2). 1 landmark at (1,1). 
```
+ 3
|_2____
```
- observations: $z = [ x+d*cos(theta + psi), y + d*sin(theta + psi)]$ 
- e_ij = z_j_hat - z_i_hat
- J_ij = `del(e_ij)/del(X)`: [... A_ij, ..., B_ij, ...]
    - `(A_ij) = del(e_ij)/del(X_i)`, 
    - `(B_ij) = del(e_ij)/del(X_j)`
- H_ij = `J_ij^T omega J_ij = [A_ij^T omega A_ij, ..., A_ij ^ T omega B_ij; ..., B_ij ^T omega A_ij, B_ij ^ T omega B_ij, ...]`
    - omega, I'm not exactly sure. I'm using `Q^-1`
- b = `[A_ij^T omega e_ij ... B_ij^T omega e_ij]`
- `del_x = H_ij^-1 b`

$$

e_ij = z_j_hat - z_i_hat, where z_i_hat = x_i + relative_pose(d,theta)
[ x_j+d_j*cos(theta_j + psi_j) - (x_i+d_i*cos(theta_i + psi_i)); 
y_j + d_j*sin(theta_j + psi_j) - (y_i + d_i*sin(theta_i + psi_i))]
$$

In [112]:
# range, bearing
Q = np.array([[0.1, 0], [0, 0.1]])

x_true = np.array([
    [0, 0, 0.0],
    [1, 0, 0],
    [1, 1, np.pi/2]
])
x_odom = copy.deepcopy(x_true)
x_odom  += np.array([
    [0.0, 0.0, 0.0],
    [0.3, 0.2, 0.2],
    [0.4, 0.3, -0.1]
])
        
# range, bearing
x_observations_ls = [
    get_observation(0, 1, x_true),
    get_observation(2, 1, x_true),
    get_observation(3, 1, x_true),
]
assert len(x_observations_ls[0]) == len(x_true) == len(x_odom)
N = len(x_true)

node_pairs = list(itertools.combinations(range(N), 2))
for _ in range(5):

    #TODO Remember to remove
    print(f'Rico: ==============')
    H = np.zeros((3 * N, 3 * N))
    b = np.zeros(3 * N)
    J = np.zeros((2, 3*N))
    for i, j in node_pairs:
        for x_obs in x_observations_ls:
            #TODO Remember to remove
            print(f'----------')
            # J_ij = [dx: ...x_i, y_i, theta_i, ... x_j, y_j, theta_j ...] 
            #  = [
            #         ...-1, 0, d_i*sin*(theta_i + psi_i)... 1, 0, -d_j*sin*(theta_j + psi_j);
            #         ...0, -1, -d_i*cos(theta_i + psi_i)... 0, 1, d_j*cos(theta_j + psi_j)
            #  = [          A_ij                                B_ij    ]
            A_ij = get_A(x_odom=x_odom, x_obs=x_obs, i=i)
            B_ij = get_B(x_odom=x_odom, x_obs=x_obs, j=j)
            e_ij = get_e(x_odom=x_odom, x_obs=x_obs, i=i, j=j) #[1x2]
            #TODO Remember to remove
            print(f'e_ij: {e_ij}')
            omega = get_omega(x_odom=x_odom, x_obs=x_obs, i=i, j=j, Q=Q)
            #TODO Remember to remove
            print(f'omega: {omega}')
            H[3*i: 3*i+3, 3*i: 3*i+3] += A_ij.T @ omega @ A_ij
            H[3*i: 3*i+3, 3*j: 3*j+3] += A_ij.T @ omega @ B_ij
            H[3*j: 3*j+3, 3*i: 3*i+3] += B_ij.T @ omega @ A_ij
            H[3*j: 3*j+3, 3*j: 3*j+3] += B_ij.T @ omega @ B_ij
            # print(f'A_ij: {A_ij}')
            # print(f'B_ij: {B_ij}')
            # print(f'Rico: H[ii]: {H[3*i: 3*i+3, 3*i: 3*i+3]}')
            # print(f'Rico: H[ij]: {H[3*i: 3*i+3, 3*j: 3*j+3]}')
            # print(f'Rico: H[ji]: {H[3*j: 3*j+3, 3*i: 3*i+3]}')
            # print(f'Rico: H[jj]: {H[3*j: 3*j+3, 3*j: 3*j+3]}')
            # print(f'Rico: {H}')
            b_i = A_ij.T @ omega @ e_ij
            b_j = B_ij.T @ omega @ e_ij
            b[3*i : 3*i +3] += b_i
            b[3*j : 3*j +3] += b_j
    H[0:3, 0:3] += 1*np.identity(3)
    

    # print(f'H: {H}')
    # print(f'H det: {np.linalg.det(H)}')
    # print(f'H========-1: {np.linalg.inv(H)}')
    # print(f'H-1*H: {np.linalg.inv(H) @ H}')
    dx = - np.linalg.inv(H) @ b
    # print(f'b: {b}')
    # print(f'dx: {dx}')
    for k in range(N):
        #TODO Remember to remove
        x_odom[k] += dx[3*k : 3*k+3]
        x_odom[k][2] = normalize_to_pi(x_odom[k][2])
    #TODO Remember to remove
    print(f'x_odom: {x_odom}')
    

Rico: ==============
----------
Rico: d_i = 1.0, psi_i = 1.5707963267948966
Rico: d_i = 1.4142135623730951, psi_i = 2.356194490192345
e_ij: [ 0.12126409 -0.01860275]
omega: [[5. 0.]
 [0. 5.]]
----------
Rico: d_i = 2.23606797749979, psi_i = 0.4636476090008061
Rico: d_i = 1.4142135623730951, psi_i = 0.7853981633974483
e_ij: [0.08139725 0.37873591]
omega: [[5. 0.]
 [0. 5.]]
----------
Rico: d_i = 3.1622776601683795, psi_i = 0.3217505543966422
Rico: d_i = 2.23606797749979, psi_i = 0.4636476090008061
e_ij: [0.06146382 0.57740524]
omega: [[ 5.00000000e+00  8.67361738e-17]
 [-8.67361738e-17  5.00000000e+00]]
----------
Rico: d_i = 1.0, psi_i = 1.5707963267948966
Rico: d_i = 1.0, psi_i = 1.5707963267948966
e_ij: [0.40499583 0.39983342]
omega: [[5. 0.]
 [0. 5.]]
----------
Rico: d_i = 2.23606797749979, psi_i = 0.4636476090008061
Rico: d_i = 1.0, psi_i = -1.5707963267948966
e_ij: [0.39500417 0.20016658]
omega: [[ 5.00000000e+00 -4.33680869e-17]
 [ 4.33680869e-17  5.00000000e+00]]
----------
Ric